In [1]:
# importar bibliotecas
import pandas as pd
import numpy as np
import datetime as dt

# ignorar as mensagens de Warning
pd.options.mode.chained_assignment = None

# 1 Ler datasets

### 1.1 casos de COVID 
### fonte: https://covid.saude.gov.br/

In [2]:
# Caminho dos datasets dos casos de covid
## 2020
df_2020_parte1 = 'HIST_PAINEL_COVIDBR_2020_Parte1_29mar2024.csv'
df_2020_parte2 = 'HIST_PAINEL_COVIDBR_2020_Parte2_29mar2024.csv'
## 2021
df_2021_parte1 = 'HIST_PAINEL_COVIDBR_2021_Parte1_29mar2024.csv'
df_2021_parte2 = 'HIST_PAINEL_COVIDBR_2021_Parte2_29mar2024.csv'
## 2022
df_2022_parte1 = 'HIST_PAINEL_COVIDBR_2022_Parte1_29mar2024.csv'
df_2022_parte2 = 'HIST_PAINEL_COVIDBR_2022_Parte2_29mar2024.csv'
## 2023
df_2023_parte1 = 'HIST_PAINEL_COVIDBR_2023_Parte1_29mar2024.csv'
df_2023_parte2 = 'HIST_PAINEL_COVIDBR_2023_Parte2_29mar2024.csv'
## 2024
df_2024_parte1 = 'HIST_PAINEL_COVIDBR_2024_Parte1_29mar2024.csv'

# colunas que serão lidas
colunas = ['regiao', 'estado', 'municipio', 'data', 'casosNovos']

# Carregar dataset dos casos covid
df1 = pd.read_csv(df_2020_parte1, sep=';', usecols=colunas)
df2 = pd.read_csv(df_2020_parte2, sep=';', usecols=colunas)
df3 = pd.read_csv(df_2021_parte1, sep=';', usecols=colunas)
df4 = pd.read_csv(df_2021_parte2, sep=';', usecols=colunas)
df5 = pd.read_csv(df_2022_parte1, sep=';', usecols=colunas)
df6 = pd.read_csv(df_2022_parte2, sep=';', usecols=colunas)
df7 = pd.read_csv(df_2023_parte1, sep=';', usecols=colunas)
df8 = pd.read_csv(df_2023_parte2, sep=';', usecols=colunas)
df9 = pd.read_csv(df_2024_parte1, sep=';', usecols=colunas)

# unir os dfs
df_casos_original = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9])

# excluir os dfs 
del df1
del df2
del df3
del df4
del df5
del df6
del df7
del df8
del df9

In [3]:
# usar só onde a coluna 'município' é 'NaN' (vai filtrar só os casos novos por UF)
casos_full_filtrado = df_casos_original[df_casos_original['municipio'].isna()]

# dropar coluna 'municipio'
casos_full_filtrado = casos_full_filtrado.drop('municipio', axis=1)

# remover onde 'regiao == Brasil'
casos_full_filtrado = casos_full_filtrado.query('regiao != "Brasil"')

# dropar colunas 'regiao'
casos_full_filtrado = casos_full_filtrado.drop('regiao', axis=1)

# total de casos
casos_full_filtrado['casosNovos'].sum()

38796900

### 1.2 Genomas depositados no GISAID

In [4]:
# Cerregar dataset com ProbWrongDates (prováveis datas de coleta erradas. fonte: Tiago)
df_wrongDate = pd.read_csv('Prob.wrongDates.toExclude.txt', sep='\t')

In [5]:
# carregar metadados do GISAID (fonte: https://gisaid.org/)
# colunas a serem lidas
colunas = ['Virus name', 'Accession ID', 'Collection date', 
           'Location', 'Pango lineage', 'Variant']

# ler dataset
gisaid_original = pd.read_csv('metadata.tsv', sep='\t', usecols=colunas)

# filtrar apenas amostras do Brasil
dados_gisaid = gisaid_original[gisaid_original['Location'].str.contains('Brazil')]

# deletar df original
del gisaid_original

# total de amostras brasileiras
print('Amostras:', dados_gisaid.shape[0])

Amostras: 252285


# 2 Pré análise dos dados

In [ ]:
# substituir espaços por underscore nos nomes das colunas
dados_gisaid.columns = dados_gisaid.columns. str. replace(' ','_')

# renomear a coluna
dados_gisaid_renomeado = dados_gisaid.rename(columns={'Pango_lineage': 'Lineage'})

In [50]:
'''
os gráficos 'genomas/laboratório' considera todos os depósitos, até os 'NAN' ou 'unassigned'
'''
# fazer uma cópia do dataset para usar nos gráficos de 'genomas/laboratório'
df_lab = dados_gisaid.copy()

In [51]:
# excluir as amostras que estão como 'Unassigned' na linhagem
dados_gisaid_renomeado = dados_gisaid_renomeado.query('Lineage != "Unassigned"')

# excluir as amostras que estão como 'NaN' na linhagem
dados_gisaid_renomeado = dados_gisaid_renomeado[dados_gisaid_renomeado['Lineage'].notna()]

In [7]:
'''
Algumas amostras vieram do gisaid classificadas erradas (deveriam ser JD.1 em vez de XBB.1.5).
Tiago quem forneceu os 'Accession IDs' das amostras 'XBB.1.5' a serem alteradas para JD.1.* 
'''
# ler o df do Tiago com os 'Accession IDs'
df_tiago = pd.read_csv('nextclade_jd.tsv', sep='\t')
# rcolcoar os Accession IDs em uma lista
lista_ID = list(df_tiago['accession'])

'''
separar o df do gisaid em 2 dfs: 
um sem as amostras do Accession ID do Tiago; 
o outro só com os Accession ID do Tiago 
'''
# df1 sem os Accession ID do Tiago
df_1 = dados_gisaid_renomeado.query('Accession_ID not in @lista_ID')
# df2 com os Accession ID do Tiago
df_2 = dados_gisaid_renomeado.query('Accession_ID in @lista_ID')

# trocar XBB.1.5 por JD.1.* no df2
df_2['Lineage'] = 'JD.1.*'
df_2['Lineage'].unique()

# unir de volta os 2 dfs: df1 + df2
dados_gisaid_renomeado_reclassificado = pd.concat([df_1, df_2])

JD.1.2      54
JD.1        42
XBB.1.5     13
JD.1.1       9
JD.1.1.1     1
Name: Lineage, dtype: int64
119


# 3 Dicionários

In [10]:
# dicionário para padronizar/corrigir os nomes dos estados
dict_nomes_corretos_estados = {
    'Brasilia': 'Distrito Federal',
    'Sao Paulo': 'São Paulo',
    'Goias': 'Goiás',
    'Espirito Santo': 'Espírito Santo',
    'Federal District': 'Distrito Federal',
    'Para': 'Pará',
    'Amapa': 'Amapá',
    'Maranhao': 'Maranhão',
    'Ceara': 'Ceará',
    'Piaui': 'Piauí',
    'Parana': 'Paraná',
    'Rondonia': 'Rondônia',
    'Paraiba': 'Paraíba',
    'Rio De Janeiro': 'Rio de Janeiro',
    'Rio Grande Do Sul': 'Rio Grande do Sul',
    'Goais': 'Goiás',
    'Porto Alegre': 'Rio Grande do Sul',
    'Guapimirim': 'Rio de Janeiro',
    'Porto Alegre':  'Rio Grande do Sul',
    'ES':  'Espírito Santo',
    'RJ': 'Rio de Janeiro',
    'PR': 'Paraná',
    'Recife': 'Pernambuco',
    'Fortaleza': 'Ceará',
    'Curitiba': 'Paraná',
    'Manaus': 'Amazonas',
    'MG': 'Minas Gerais',
    'Rcre': 'Acre',
    'RN': 'Rio Grande do Norte'}

# dicionário para criar as siglas dos Estados
dict_estado_para_sigla = {
    'São Paulo': 'SP', 
    'Paraíba': 'PB', 
    'Sergipe': 'SE',
    'Piauí': 'PI',
    'Pará': 'PA',
    'Maranhão': 'MA',
    'Acre': 'AC',
    'Amapá': 'AP',
    'Ceará': 'CE',
    'Espírito Santo': 'ES',
    'Rio Grande do Sul': 'RS',
    'Minas Gerais': 'MG',
    'Santa Catarina': 'SC',
    'Roraima': 'RR',
    'Alagoas': 'AL',
    'Pernambuco': 'PE',
    'Rio Grande do Norte': 'RN',
    'Mato Grosso': 'MT',
    'Paraná': 'PR',
    'Mato Grosso do Sul': 'MS',
    'Distrito Federal': 'DF',
    'Goiás': 'GO',
    'Bahia': 'BA',
    'Amazonas': 'AM',
    'Rondônia': 'RO',
    'Rio de Janeiro': 'RJ',
    'Tocantins': 'TO'}

# mapear o Estado com sua respectiva região 
dict_sigla_para_regiao = {
    "AC": "Norte", "AM": "Norte", "RR": "Norte", "RO": "Norte", 
    "PA": "Norte", "AP": "Norte", "TO": "Norte", 
    "MA": "Nordeste", "PI": "Nordeste", "CE": "Nordeste", 
    "RN": "Nordeste", "PB": "Nordeste", "PE": "Nordeste",
    "AL": "Nordeste", "SE": "Nordeste", "BA": "Nordeste",
    "MT": "Centro-oeste", "MS": "Centro-oeste", 
    "GO": "Centro-oeste", "DF": "Centro-oeste",
    "SP": "Sudeste", "RJ": "Sudeste",
    "MG": "Sudeste", "ES": "Sudeste",
    "PR": "Sul", "SC": "Sul",
    "RS": "Sul"}

# dicionário para criar os nomes dos Estados a partir da sigle 
# para o df dos Casos de Covid 
dict_sigla_para_estado = {
    'SP': 'São Paulo', 
    'PB': 'Paraíba', 
    'SE': 'Sergipe',
    'PI': 'Piauí',
    'PA': 'Pará',
    'MA': 'Maranhão',
    'AC': 'Acre',
    'AP': 'Amapá',
    'CE': 'Ceará',
    'ES': 'Espírito Santo',
    'RS': 'Rio Grande do Sul',
    'MG': 'Minas Gerais',
    'SC': 'Santa Catarina',
    'RR': 'Roraima',
    'AL': 'Alagoas',
    'PE': 'Pernambuco',
    'RN': 'Rio Grande do Norte',
    'MT': 'Mato Grosso',
    'PR': 'Paraná',
    'MS': 'Mato Grosso do Sul',
    'DF': 'Distrito Federal',
    'GO': 'Goiás',
    'BA': 'Bahia',
    'AM': 'Amazonas',
    'RO': 'Rondônia',
    'RJ': 'Rio de Janeiro',
    'TO': 'Tocantins'}

# 4 Funções

In [69]:
'''
Remove as entradas duplicadas de acordo com o 'Virus name'
(Essa função talvez não seja mais necessária)
'''
def remove_duplicatas(df):
  # pegar o índice das linhas duplicadas
  duplicatas_virus_name = df[df['Virus_name'].duplicated()].index
  # excluir essas linhas pelo índice
  df2 = df.drop(duplicatas_virus_name)
  # pegar o índice das linhas duplicadas
  duplicatas_accession_id = df2[df2['Accession_ID'].duplicated()].index
  df2 = df2.drop(duplicatas_accession_id)
  print('Total de linhas duplicadas removidas:', df.shape[0]-df2.shape[0], 'Done!')
  return df2


'''
Remove as amostras com ProbWrongDate (datas de coleta provavelmente erradas)
'''
def remove_wrondDate(df, df_wrongDate):  
  # lista com os valores da coluna 'Virus name' das amostras do dataset 'Prob.wrongDates.toExclude' 
  wrongDate_virus_names = df_wrongDate["Virus.name"].values
  # remover as amostras com ProbWrongDates dos dados do GISAID
  df2 = df.query("Virus_name not in @wrongDate_virus_names")
  print('Total de linhas com ProbWrongDate removidas:', df.shape[0]-df2.shape[0], 'Done!')
  return df2 


'''
Extrai o Estado das informações contidas na variável 'Location'.
Retorna o mesmo df com uma nova variável 'Estado'.
Retorna NaN caso não haja a informação do Estado.
'''
def cria_estado(df):
  # função lambda para extrair o elemento do índice 2 (que é o Estado) de cada linha
  # retorna 'NaN' caso não tenha o Estado no índice 2
  extrai = lambda lista: lista[2] if len(lista) >= 3 else np.NaN
  # cria a coluna 'Estado' e aplica a função lambda 'extrai'
  df['Estado'] = df['Location'].str.split('/').apply(extrai)
  # remover espaços iniciais e finais que possam estar no nome do Estado
  df['Estado'] = df['Estado'].str.strip()
  print(df['Estado'].unique())
  print('Estados únicos =', df['Estado'].nunique(), 'Done!')
  return df


'''
Corrige os nomes dos Estados
'''
def corrige_nomes_estados(df, dict_nomes_corretos_estados):
  # substituir pelos nomes padronizados
  df['Estado'] = df['Estado'].replace(dict_nomes_corretos_estados)
  print(df['Estado'].unique())
  print('Estados únicos - corrigidos =', df['Estado'].nunique(), 'Done!')
  return df


''''
Cria as colunas 'UF' e 'Região'
'''
def cria_siglas_e_regiao(df, dict_siglas_estados, dict_regiao):
  # criar a coluna 'UF' a partir do 'Estado'
  df['UF'] = df['Estado'].replace(dict_siglas_estados)
   # criar a coluna 'Região' a partir da 'UF'
  df['Região'] = df['UF'].map(dict_regiao)  
  print('UFs únicas', df['UF'].nunique())
  print('Regiões únicas', df['Região'].nunique(), 'Done!') 
  return df


'''
Extrai as informações de mês e ano (= fperíodo) da coluna 'Collection date' 
'''
def cria_mes_ano(df):
  # converter coluna para datetime
  df['Collection_date'] = pd.to_datetime(df['Collection_date'])    
  # criar a variável 'Mes'
  df['Mes'] = df['Collection_date'].dt.month 
  # criar a variável 'Ano'
  df['Ano'] = df['Collection_date'].dt.year 
  # criar a data ormato 'ANO-MES'
  df['Data'] = (df['Ano']).astype('str') + '-' + (df['Mes']).astype('str')
  df['Data'] = pd.to_datetime(df[f'Data'])
  # dicionário com o número do mês e seu nome (3 letras iniciais)
  dict_mes = {1: 'Jan', 2: 'Fev', 3: 'Mar', 4: 'Abr', 5: 'Mai', 6: 'Jun',
              7: 'Jul', 8: 'Ago', 9: 'Set', 10: 'Out', 11: 'Nov', 12: 'Dez'}
  # criar a coluna 'Mes_nome' com o nome do mês
  df['Mes_nome'] = df['Mes'].map(dict_mes)
  # concatenar mês + ano como strings
  df['Período'] = df['Mes_nome'] + ' ' + (df['Ano']).astype('str')
  # remover a coluna 'Mes_nome'
  df.drop('Mes_nome', axis=1, inplace=True)
  print('Collection_date: datas extraídas. Done!')
  return df


'''
Identifica laboratório: sequências da 'Fiocrus' e 'Outros'
'''
def identifica_lab(df):
  # colocar tudo em minúsculo
  df['Virus_name'] = df['Virus_name'].str.lower()
  # procura as linhas que tem 'fiocruz' na coluna 'Virus_name'; 
  # recebe True se tiver 'fiocruz'; recebe False caso contrário
  df['Laboratório'] = df['Virus_name'].str.contains("fiocruz")
  # dicionário para substituir o True e o False
  lab = {True: 'FIOCRUZ',
         False: 'Outros'}  
  # criar a variável 'Laboratório' que vai receber de qual lab o vírus foi sequenciado      
  # substituir True e False por 'FIOCRUZ' e 'Outros'
  df['Laboratório'] = df['Laboratório'].map(lab)  
  print('Labs identificados. Done!')
  return df


'''
Linhagens: para os gráficos do final do dashboard (série histórica).
Vai agrupar as sub-linhagens em sua 'linhagem-mãe' (o asterisco * indica que há sub-linhagens)
'''
def cria_linhagem(df):
  # remover, pelo índice, as 'P.1' que foram classificadas erradas no começo da pandemia
  indices_P1 = df[df['Lineage'].str.startswith('P.1')].query('Ano == "2020" and Mes in ["09", "10", "11"]').index
  df = df.drop(indices_P1)
  # classificar as 'P.1.1' e 'P.1.2' como 'P.1'
  df['Lineage'] = df['Lineage'].apply(lambda x: 'P.1' if (x == 'P.1.1' or x == 'P.1.2') else x)

  # Delta
  df['Linhagem'] = df['Lineage'].apply(lambda x: 'B.1.617.2+AY.* (Delta)' if (x.startswith('AY.') or x == 'B.1.617.2') else x)
  # Alfa
  df['Linhagem'] = df['Linhagem'].apply(lambda x: 'B.1.1.7 (Alfa)' if (x.startswith('Q') or x == 'B.1.1.7') else x)
  # Beta
  df['Linhagem'] = df['Linhagem'].apply(lambda x: 'B.1.351 (Beta)' if x.startswith('B.1.351') else x)
  # Gama
  df['Linhagem'] = df['Linhagem'].apply(lambda x: 'P.1.* (Gama)' if x.startswith('P.1') else x)
  # Omicron
  df['Linhagem'] = df['Linhagem'].apply(lambda x: 'BA.1.* (Omicron)' if x.startswith('BA.1') else x)
  df['Linhagem'] = df['Linhagem'].apply(lambda x: 'BA.2.* (Omicron)' if x.startswith('BA.2') else x)
  df['Linhagem'] = df['Linhagem'].apply(lambda x: 'BA.4.* (Omicron)' if x.startswith('BA.4') else x)
  df['Linhagem'] = df['Linhagem'].apply(lambda x: 'BA.5.* (Omicron)' if (x.startswith('BA.5') or\
                                                                         x.startswith('BE') or\
                                                                         x.startswith('BQ') or\
                                                                         x.startswith('DL')) else x)
  ## XBB.1.5.70 - não agrupar essa sub-linhagem com as demais XBB.*
  xbb70 = df[df['Lineage'].str.startswith('XBB.1.5.70')]['Lineage'].unique()
  df['Linhagem'] = df['Linhagem'].apply(lambda x: 'XBB.* (Omicron)' if (x.startswith('XBB') and x not in xbb70) else x) 
  df['Linhagem'] = df['Linhagem'].apply(lambda x: 'FE.1.* (Omicron)' if x.startswith('FE.1') else x)  
  df['Linhagem'] = df['Linhagem'].apply(lambda x: 'XBB.1.5.70.*+GK.* (Omicron)' if (x.startswith('XBB.1.5.70') or x.startswith('GK')) else x)  
  df['Linhagem'] = df['Linhagem'].apply(lambda x: 'JD.1.* (Omicron)' if (x.startswith('JD.1')) else x) 
  df['Linhagem'] = df['Linhagem'].apply(lambda x: 'JN.1.*+BA.2.86.* (Omicron)' if (x.startswith('JN.1') or x.startswith('BA.2.86.')) else x) 
  return df

### 4.1 Chamar funções

In [52]:
# remover duplicatas
dados = remove_duplicatas(dados_gisaid_renomeado_reclassificado)

# remover amostras com prováveis datas de coleta erradas
dados = remove_wrondDate(dados, df_wrongDate)

# criar coluna 'Estado'
dados = cria_estado(dados)

Total de linhas duplicadas removidas: 1 Done!
Total de linhas com ProbWrongDate removidas: 24 Done!
['Rio de Janeiro' 'Sao Paulo' 'Minas Gerais' 'Piaui' 'Pernambuco' 'Bahia'
 'Ceara' 'Amazonas' 'Santa Catarina' 'Rondonia' 'Mato Grosso do Sul'
 'Rio Grande do Sul' 'Goias' 'Parana' 'Para' 'Paraiba' 'Espirito Santo'
 'Acre' 'Alagoas' 'Tocantins' 'Rio Grande do Norte' 'Sergipe' 'Amapa' nan
 'Federal District' 'Mato Grosso' 'Maranhao' 'Roraima' 'RN' 'RJ']
Estados únicos = 29 Done!


In [53]:
# corrigir nomes dos Estados que vieram errados do gisaid
dados = corrige_nomes_estados(dados, dict_nomes_corretos_estados)

# criar as colunas 'UF' e 'Região'
dados = cria_siglas_e_regiao(dados, dict_estado_para_sigla, dict_sigla_para_regiao)

# criar colunas com 'mes', 'ano', 'período'
dados = cria_mes_ano(dados)

# criar a coluna 'Laboratório'
dados = identifica_lab(dados)

['Rio de Janeiro' 'São Paulo' 'Minas Gerais' 'Piauí' 'Pernambuco' 'Bahia'
 'Ceará' 'Amazonas' 'Santa Catarina' 'Rondônia' 'Mato Grosso do Sul'
 'Rio Grande do Sul' 'Goiás' 'Paraná' 'Pará' 'Paraíba' 'Espírito Santo'
 'Acre' 'Alagoas' 'Tocantins' 'Rio Grande do Norte' 'Sergipe' 'Amapá' nan
 'Distrito Federal' 'Mato Grosso' 'Maranhão' 'Roraima']
Estados únicos - corrigidos = 27 Done!
UFs únicas 27
Regiões únicas 5 Done!
Collection_date: datas extraídas. Done!
Labs identificados. Done!


# 5 Análises - gráficos Genomas sequenciados/Laboratório

In [54]:
# chamar novamente algumas das funções, porém agora é no dataset que copiamos no início
df_lab = remove_duplicatas(df_lab)
df_lab = remove_wrondDate(df_lab, df_wrongDate)
df_lab = cria_mes_ano(df_lab)
df_lab = identifica_lab(df_lab)

Total de linhas duplicadas removidas: 1 Done!
Total de linhas com ProbWrongDate removidas: 24 Done!
Collection_date: datas extraídas. Done!
Labs identificados. Done!


In [55]:
# agrupar por 'Laboratório' e 'Data' e fazer a contagem
agrupado_laboratorio = df_lab[['Virus_name', 'Laboratório', 'Data', 'Período']]\
.groupby(['Data', 'Laboratório', 'Período']).count().reset_index()

# renomear colunas
agrupado_laboratorio.rename(columns={'Virus_name': 'Quantidade'}, inplace=True)
agrupado_laboratorio.tail(4)

,Data,Laboratório,Período,Quantidade
97,2024-01-01,Outros,Jan 2024,770
98,2024-02-01,FIOCRUZ,Fev 2024,246
99,2024-02-01,Outros,Fev 2024,366
100,2024-03-01,Outros,Mar 2024,21


In [56]:
'''
USAR APENAS QUANDO NÃO HOUVER AMOSTRAS DA FIOCRUZ DEPOSITADAS NO ÚLTIMO MÊS, MAS HÁ SOMENTE AMOSTRAS DO 'OUTROS'
O NÚMERO DO ÍNDICE PRECISA SER ATUALIZADO CONFORME NOVOS MESES SÃO ADICIONADOS AOS DADOS
'''
# adiciona a última linha no final do próprio df 
agrupado_laboratorio = agrupado_laboratorio.append(agrupado_laboratorio.iloc[100], ignore_index=True)
agrupado_laboratorio.tail()

,Data,Laboratório,Período,Quantidade
97,2024-01-01,Outros,Jan 2024,770
98,2024-02-01,FIOCRUZ,Fev 2024,246
99,2024-02-01,Outros,Fev 2024,366
100,2024-03-01,Outros,Mar 2024,21
101,2024-03-01,Outros,Mar 2024,21


In [57]:
'''
USAR APENAS QUANDO NÃO HOUVER AMOSTRAS DA FIOCRUZ DEPOSITADAS NO ÚLTIMO MÊS, MAS HÁ SOMENTE AMOSTRAS DO 'OUTROS'
O NÚMERO DO ÍNDICE PRECISA SER ATUALIZADO CONFORME NOVOS MESES SÃO ADICIONADOS AOS DADOS
'''
# substitui o nome do lab por 'FIOCRUZ'
agrupado_laboratorio.loc[101, 'Laboratório'] = 'FIOCRUZ'
# substitui a quantidade de genoma depositado por zero
agrupado_laboratorio.loc[101, 'Quantidade'] = 0
agrupado_laboratorio.tail()

,Data,Laboratório,Período,Quantidade
97,2024-01-01,Outros,Jan 2024,770
98,2024-02-01,FIOCRUZ,Fev 2024,246
99,2024-02-01,Outros,Fev 2024,366
100,2024-03-01,Outros,Mar 2024,21
101,2024-03-01,FIOCRUZ,Mar 2024,0


In [58]:
# calcular valores acumulados
acumulado = agrupado_laboratorio.set_index('Laboratório').groupby(level=0).cumsum()['Quantidade']

# criar a coluna 'Acumulado' dos valores acumulados
agrupado_laboratorio['Acumulado'] = acumulado.values
agrupado_laboratorio.tail(4)

,Data,Laboratório,Período,Quantidade,Acumulado
98,2024-02-01,FIOCRUZ,Fev 2024,246,83413
99,2024-02-01,Outros,Fev 2024,366,168826
100,2024-03-01,Outros,Mar 2024,21,168847
101,2024-03-01,FIOCRUZ,Mar 2024,0,83413


In [59]:
# total de genomas depositados
agrupado_laboratorio['Quantidade'].sum()

252260

In [22]:
# traduzir: EN e ES
dic_regex_EN = {'Fev': 'Feb', 'Abr': 'Apr', 'Mai': 'May',
                'Ago': 'Aug', 'Set': 'Sep', 'Out': 'Oct', 'Dez': 'Dec'}

dic_regex_ES = {'Jan': 'Ene', 'Fev': 'Feb', 'Mai': 'May',
                'Set': 'Sep', 'Out': 'Oct', 'Dez': 'Dic'}

agrupado_laboratorio_EN = agrupado_laboratorio.rename(columns={'Data': 'Date',
                                                               'Laboratório': 'Laboratory',
                                                               'Período': 'Period',
                                                               'Quantidade': 'Quantity',
                                                               'Acumulado': 'Cumullative'})

agrupado_laboratorio_ES = agrupado_laboratorio.rename(columns={'Data': 'Fecha',
                                                               'Laboratório': 'Laboratorio',
                                                               'Quantidade': 'Cantidad'})

agrupado_laboratorio_EN = agrupado_laboratorio_EN.replace({'Outros': 'Others'})
agrupado_laboratorio_EN = agrupado_laboratorio_EN.replace(regex=dic_regex_EN)

agrupado_laboratorio_ES = agrupado_laboratorio_ES.replace({'Outros': 'Otros'})
agrupado_laboratorio_ES = agrupado_laboratorio_ES.replace(regex=dic_regex_ES)

In [23]:
# exportar
agrupado_laboratorio.to_csv('dfGenomasLaboratorio_PT.csv', index=False)
agrupado_laboratorio_EN.to_csv('dfGenomasLaboratorio_EN.csv', index=False)
agrupado_laboratorio_ES.to_csv('dfGenomasLaboratorio_ES.csv', index=False)

# 6 Análises - gráficos das variantes mais recentes (a partir de Jan 2023)

In [60]:
# filtrar amostras coletadas a partir de 2023
dados_voc = dados.query('Ano in [2023, 2024]')
dados_voc['Período'].value_counts()

Fev 2023    2655
Jan 2023    2239
Out 2023    2184
Nov 2023    1705
Set 2023    1693
Dez 2023    1684
Mar 2023    1663
Jan 2024    1402
Abr 2023    1308
Mai 2023    1241
Jun 2023     939
Ago 2023     803
Fev 2024     612
Jul 2023     442
Mar 2024      21
Name: Período, dtype: int64

In [63]:
# remover onde 'Lineage' é None
dados_voc = dados_voc[dados_voc['Lineage'] != 'None']
# remover onde 'Lineage' é NaN
dados_voc = dados_voc[dados_voc['Lineage'].notna()]

# criar a coluna 'Variante' para trabalhar nela - deixar a 'Lineage' com os dados originais
dados_voc['Variante'] = dados_voc['Lineage']

In [26]:
# salvar em listas as linhagens e suas sub-linhagens (essas serão mostradas nos gráficos)
xbb = dados_voc[dados_voc['Variante'].str.startswith('XBB.1.5')]['Variante'].unique()
gk = dados_voc[dados_voc['Variante'].str.startswith('GK')]['Variante'].unique()
fe = dados_voc[dados_voc['Variante'].str.startswith('FE.1')]['Variante'].unique()
ha = dados_voc[dados_voc['Variante'].str.startswith('HA.1')]['Variante'].unique()
eg = dados_voc[dados_voc['Variante'].str.startswith('EG.5')]['Variante'].unique()
jd = dados_voc[dados_voc['Variante'].str.startswith('JD.1')]['Variante'].unique()
ba = dados_voc[dados_voc['Variante'].str.startswith('BA.2.86.')]['Variante'].unique()
jn = dados_voc[dados_voc['Variante'].str.startswith('JN.1')]['Variante'].unique()

# colocar os valores em apenas uma lista
variantes_relevantes = [*xbb, *fe, *ha, *eg, *jd, *gk, *jn, *ba]  

# criar coluna 'Variantes relevantes' 
# 'Variantes relevantes' são mostradas no gráfico
# Outras variantes entram como 'Outras'
dados_voc['Variantes relevantes'] = dados_voc['Variante'].apply(lambda x: x if (x in variantes_relevantes) else 'Outras')

# separar a XBB.1.5.70 e suas sub-linhagens das demais XBB.1.5
xbb_1_5_70 = dados_voc[dados_voc['Variante'].str.startswith('XBB.1.5.70')]['Variante'].unique()
excluir = [*xbb_1_5_70]

# XBB.1.5* (não incluir a XBB.1.5.70 e suas sub-linhagens)
dados_voc['Variantes relevantes'] = dados_voc['Variantes relevantes'].apply(lambda x: 'XBB.1.5.* (Omicron)' if (x.startswith('XBB.1.5') and x not in excluir) else x) 
# XBB.1.5.70.*+GK.*
dados_voc['Variantes relevantes'] = dados_voc['Variantes relevantes'].apply(lambda x: 'XBB.1.5.70.*+GK.* (Omicron)' if (x.startswith('XBB.1.5.70') or x.startswith('GK')) else x)  
# HA.1.* 
dados_voc['Variantes relevantes'] = dados_voc['Variantes relevantes'].apply(lambda x: 'HA.1.* (Omicron)' if (x.startswith('HA.1')) else x) 
# FE.1.* 
dados_voc['Variantes relevantes'] = dados_voc['Variantes relevantes'].apply(lambda x: 'FE.1.* (Omicron)' if (x.startswith('FE.1')) else x) 
# EG.5.* 
dados_voc['Variantes relevantes'] = dados_voc['Variantes relevantes'].apply(lambda x: 'EG.5.* (Omicron)' if (x.startswith('EG.5')) else x) 
# JD.1.* 
dados_voc['Variantes relevantes'] = dados_voc['Variantes relevantes'].apply(lambda x: 'JD.1.* (Omicron)' if (x.startswith('JD.1')) else x) 
# JN.1 
dados_voc['Variantes relevantes'] = dados_voc['Variantes relevantes'].apply(lambda x: 'JN.1.*+BA.2.86.* (Omicron)' if (x.startswith('JN.1') or x.startswith('BA.2.86.')) else x) 

dados_voc['Variantes relevantes'].value_counts()      

Outras                         6785
XBB.1.5.70.*+GK.* (Omicron)    3530
JD.1.* (Omicron)               3158
XBB.1.5.* (Omicron)            2680
JN.1.*+BA.2.86.* (Omicron)     2365
FE.1.* (Omicron)               1885
EG.5.* (Omicron)                148
HA.1.* (Omicron)                 40
Name: Variantes relevantes, dtype: int64

In [27]:
# selecionar apenas algumas colunas
agrupado_voc = dados_voc[['Virus_name', 'Região', 'Estado', 'Data', 
                          'Período', 'Variantes relevantes']]

# agrupar pelo período, região, variante, lab e contar as freqs
agrupado_voc = agrupado_voc.groupby(['Data', 'Período', 'Região', 'Estado', 
                                     'Variantes relevantes']).count()

# resetar índice
agrupado_voc = agrupado_voc.reset_index()

# renomear colunas
agrupado_voc = agrupado_voc.rename(columns={'Virus_name': 'Quantidade'})
agrupado_voc.head(2)

,Data,Período,Região,Estado,Variantes relevantes,Quantidade
0,2023-01-01,Jan 2023,Centro-oeste,Distrito Federal,Outras,4
1,2023-01-01,Jan 2023,Centro-oeste,Goiás,Outras,107


In [28]:
# traduzir EN e ES
dic_replace_regiao_EN = {'Norte': 'North', 
                         'Nordeste': 'Northeast',
                         'Sudeste': 'Southeast', 
                         'Sul': 'South',
                         'Centro-oeste': 'Central-west',
                         'Outras': 'Others'}

dic_regex_EN = {'Fev': 'Feb', 'Abr': 'Apr', 'Mai': 'May',
                'Ago': 'Aug', 'Set': 'Sep', 'Out': 'Oct',
                'Dez': 'Dec'}

dic_replace_regiao_ES = {'Nordeste': 'Noreste',
                         'Sudeste': 'Sureste', 
                         'Sul': 'Sur',
                         'Outras': 'Otras'}

dic_regex_ES = {'Jan': 'Ene', 'Fev': 'Feb', 'Mai': 'May',
                'Set': 'Sep', 'Out': 'Oct', 'Dez': 'Dic'}    

agrupado_voc_EN = agrupado_voc.replace(dic_replace_regiao_EN)
agrupado_voc_EN = agrupado_voc_EN.replace(regex=dic_regex_EN)
agrupado_voc_EN = agrupado_voc_EN.rename(columns={'Data': 'Date', 'Período': 'Period',
                                                  'Região': 'Region', 'Estado': 'State',
                                                  'Variante': 'Variant', 
                                                  'Variantes relevantes': 'Relevant variants',
                                                  'Quantidade': 'Quantity'})

agrupado_voc_ES = agrupado_voc.replace(dic_replace_regiao_ES)
agrupado_voc_ES = agrupado_voc_ES.replace(regex=dic_regex_ES)
agrupado_voc_ES = agrupado_voc_ES.rename(columns={'Data': 'Fecha', 
                                                  'Região': 'Región', 
                                                  'Quantidade': 'Cantidad'})            

In [29]:
# exportar
agrupado_voc.to_csv('dfVariantes_PT.csv', index=False)
agrupado_voc_EN.to_csv('dfVariantes_EN.csv', index=False)
agrupado_voc_ES.to_csv('dfVariantes_ES.csv', index=False)

### 6.1 Variantes mais frequentes do último mês (parte superior direita do dashboard)

In [ ]:
'''
Atualizar o valor do período para o mês mais recente
'''
# criar um df só com os depósitos do mês mais recente
dfLinMes_PT = agrupado_voc.query('Período == "Mar 2024"')[['Variantes relevantes', 'Quantidade']]

# renomar coluna
dfLinMes_PT = dfLinMes_PT.reset_index().rename(columns={'Quantidade': 'Frequencia'})

# calcular a porcentagem de cada variante em relação ao total do mês
dfLinMes_PT['Porcentagem'] = round((dfLinMes_PT['Frequencia'] / dfLinMes_PT['Frequencia'].sum())*100,1)

# classificar do maior pro menor
dfLinMes_PT = dfLinMes_PT.sort_values('Porcentagem', ascending=False)

# dropar índice
dfLinMes_PT = dfLinMes_PT.drop('index', axis=1)

# traduzir
dfLinMes_EN = dfLinMes_PT.replace({'Outras': 'Others'})
dfLinMes_ES = dfLinMes_PT.replace({'Outras': 'Otras'})


# exportar
dfLinMes_PT[:3].to_csv('dfLinhagensMes_PT.csv', index=False)
dfLinMes_EN[:3].to_csv('dfLinhagensMes_EN.csv', index=False)
dfLinMes_ES[:3].to_csv('dfLinhagensMes_ES.csv', index=False)

# 7 Análises - gráficos da série histórica

In [100]:
# chamar função para criar as linhagens da série histórica
dados_lin = cria_linhagem(dados)

# excluir Jan 2020 - começo da pandemia, pouquíssimas amostras (distorce o gráfico)
dados_lin = dados_lin.query('Período != "Jan 2020"')

In [101]:
# Linhagens mais relevantes serão mostradas nos gráficos
linhagens_relevantes = ['B.1.1', 'B.1.1.28', 'B.1.1.33', 'P.2',
                        'B.1.1.7 (Alfa)', 
                        'B.1.617.2+AY.* (Delta)', 
                        'P.1.* (Gama)', 
                        'BA.1.* (Omicron)', 'BA.2.* (Omicron)', 
                        'BA.4.* (Omicron)', 'BA.5.* (Omicron)',
                        'XBB.* (Omicron)', 'FE.1.* (Omicron)',
                        'XBB.1.5.70.*+GK.* (Omicron)',
                        'JD.1.* (Omicron)',
                        'JN.1.*+BA.2.86.* (Omicron)']

# criar uma coluna 'Linhagens relevantes' ; quem não for relevante, será classificada como 'Outras'
dados_lin['Linhagens relevantes'] = dados_lin['Linhagem'].apply(lambda x: x if x in linhagens_relevantes else 'Outras')
dados_lin['Linhagens relevantes'].value_counts() 

P.1.* (Gama)                   58075
BA.1.* (Omicron)               49880
B.1.617.2+AY.* (Delta)         48690
BA.5.* (Omicron)               34138
BA.2.* (Omicron)               14518
Outras                          7130
B.1.1.28                        5598
BA.4.* (Omicron)                5382
XBB.* (Omicron)                 5178
P.2                             4913
B.1.1.33                        3933
XBB.1.5.70.*+GK.* (Omicron)     3530
JD.1.* (Omicron)                3158
JN.1.*+BA.2.86.* (Omicron)      2326
FE.1.* (Omicron)                1886
B.1.1.7 (Alfa)                  1401
B.1.1                           1015
Name: Linhagens relevantes, dtype: int64

In [102]:
# selecionar apenas algumas colunas
agrupado_linhagem = dados_lin[['Virus_name', 'Data', 'Período', 'Região', 'Estado', 'Laboratório', 'Linhagens relevantes']]

# Agrupar - Não incluir 'Linhagem' senão ficará uma tabela enorme para exportar
agrupado_linhagem = agrupado_linhagem.groupby(['Data', 'Período', 'Região', 
                                               'Estado', 'Laboratório', 'Linhagens relevantes']).count()

# resetar índice
agrupado_linhagem = agrupado_linhagem.reset_index()

# # renomear colunas
agrupado_linhagem = agrupado_linhagem.rename(columns={'Virus_name': 'Quantidade'})
agrupado_linhagem.head(2)

,Data,Período,Região,Estado,Laboratório,Linhagens relevantes,Quantidade
0,2020-02-01,Fev 2020,Nordeste,Bahia,Outros,B.1.1,1
1,2020-02-01,Fev 2020,Sudeste,Espírito Santo,FIOCRUZ,Outras,1


In [36]:
# traduzir EN e ES
dic_replace_regiao_EN = {'Norte': 'North', 
                         'Nordeste': 'Northeast',
                         'Sudeste': 'Southeast', 
                         'Sul': 'South',
                         'Centro-oeste': 'Central-west',
                         'Outras': 'Others',
                         'Outros': 'Others'}

dic_regex_EN = {'Fev': 'Feb', 'Abr': 'Apr', 'Mai': 'May',
                'Ago': 'Aug', 'Set': 'Sep', 'Out': 'Oct',
                'Dez': 'Dec', 
                'Gama': 'Gamma', 'Alfa': 'Alpha'}

dic_replace_regiao_ES = {'Nordeste': 'Noreste',
                         'Sudeste': 'Sureste', 
                         'Sul': 'Sur',
                         'Outras': 'Otras',
                         'Outros': 'Otros'}

dic_regex_ES = {'Jan': 'Ene', 'Fev': 'Feb', 'Mai': 'May',
                'Set': 'Sep', 'Out': 'Oct', 'Dez': 'Dic'}

agrupado_linhagem_EN = agrupado_linhagem.replace(dic_replace_regiao_EN)
agrupado_linhagem_EN = agrupado_linhagem_EN.replace(regex=dic_regex_EN)
agrupado_linhagem_EN = agrupado_linhagem_EN.rename(columns={'Data': 'Date', 'Período': 'Period',
                                                            'Região': 'Region', 'Estado': 'State',
                                                            'Variante': 'Variant', 
                                                            'Laboratório': 'Laboratory',
                                                            'Linhagens relevantes': 'Relevant lineages',
                                                            'Quantidade': 'Quantity'})

agrupado_linhagem_ES = agrupado_linhagem.replace(dic_replace_regiao_ES)
agrupado_linhagem_ES = agrupado_linhagem_ES.replace(regex=dic_regex_ES)
agrupado_linhagem_ES = agrupado_linhagem_ES.rename(columns={'Data': 'Fecha',
                                                            'Região': 'Región',
                                                            'Laboratório': 'Laboratorio',
                                                            'Linhagens relevantes': 'Lineajes relevantes',
                                                            'Quantidade': 'Cantidad'})

In [37]:
# exportar
agrupado_linhagem.to_csv("dfLinhagens_PT.csv", index=False)
agrupado_linhagem_EN.to_csv("dfLinhagens_EN.csv", index=False)
agrupado_linhagem_ES.to_csv("dfLinhagens_ES.csv", index=False)

# 8 Mapas - Genomas/Casos confirmados 

### 8.1 Genomas

In [106]:
# copiar df
dados2 = dados.copy()
# remover onde 'Estado == NaN'
dados2 = dados2[dados2['Estado'].notna()]

# selecionar apenas algumas colunas
df_genomas = dados2[["Virus_name", "Data", "Período", "Estado"]]

# agrupar Genomas pela 'data' e pelo 'Estado' e fazer a contagem; resetar índice
df_genomas = df_genomas.groupby(["Data", "Período", "Estado"]).count().reset_index()

# renomear coluna
df_genomas = df_genomas.rename(columns={"Virus_name": "Genomas sequenciados"})

In [107]:
# agrupar pelo Estado = total de genomas/Estado
df_genomas = df_genomas.groupby("Estado").sum().reset_index()
df_genomas

,Estado,Genomas sequenciados
0,Acre,1607
1,Alagoas,3278
2,Amapá,1302
3,Amazonas,11787
4,Bahia,10398
5,Ceará,11443
6,Distrito Federal,3475
7,Espírito Santo,4579
8,Goiás,9187
9,Maranhão,1407


### 8.2 Casos covid

In [105]:
# renomear colunas
casos_full_filtrado = casos_full_filtrado.rename(columns={'casosNovos': 'Casos',
                                                          'estado': 'UF'})

# Somar total de casos por 'UF'
df_casos = casos_full_filtrado.groupby('UF').sum().reset_index()

# criar a coluna 'Estado'
df_casos['Estado'] = df_casos['UF'].map(dict_sigla_para_estado)
df_casos.head(3)

,UF,Casos,Estado
0,AC,168785,Acre
1,AL,346836,Alagoas
2,AM,641988,Amazonas


### 8.3 Unir os dfs de Genomas e Casos (unir pelo 'Estado')

In [108]:
# merge com o dataframe que tem a quantidade total de genomas/estado 
df_casos_genomas = pd.merge(df_casos, df_genomas, on='Estado')
df_casos_genomas

,UF,Casos,Estado,Genomas sequenciados
0,AC,168785,Acre,1607
1,AL,346836,Alagoas,3278
2,AM,641988,Amazonas,11787
3,AP,191454,Amapá,1302
4,BA,1861497,Bahia,10398
5,CE,1522256,Ceará,11443
6,DF,942041,Distrito Federal,3475
7,ES,1391158,Espírito Santo,4579
8,GO,2030728,Goiás,9187
9,MA,500694,Maranhão,1407


In [109]:
# dividir os casos confirmados por 100.000
df_casos_genomas['casos_dividido_por_100k'] = df_casos_genomas['Casos']/10**5

# dividir o total de genomas por casos_dividido_por_100k
resultado = df_casos_genomas['Genomas sequenciados'] / df_casos_genomas['casos_dividido_por_100k'] 

# arredondar para 1 casa decimal
df_casos_genomas['Genomas/100 mil casos'] = round(resultado, 1)
df_casos_genomas.head(2)

,UF,Casos,Estado,Genomas sequenciados,casos_dividido_por_100k,Genomas/100 mil casos
0,AC,168785,Acre,1607,1.68785,952.1
1,AL,346836,Alagoas,3278,3.46836,945.1


In [43]:
# traduzir EN e ES
df_casos_genomas_EN = df_casos_genomas.rename(columns={'Estado': 'State', 
                                                       'Genomas sequenciados': 'Genomes sequenced',
                                                       'Genomas/100 mil casos': 'Genomes/100K cases', 
                                                       'Casos': 'Cases'})

df_casos_genomas_ES = df_casos_genomas.rename(columns={'Genomas sequenciados': 'Genomas secuenciados'})

In [44]:
# exportar
df_casos_genomas.to_csv('dfMapa_PT.csv', index=False)
df_casos_genomas_EN.to_csv('dfMapa_EN.csv', index=False)
df_casos_genomas_ES.to_csv('dfMapa_ES.csv', index=False)